# First peek: raw nRF log and HEX strings

Load the first 100 lines of a log in `data/raw/` and show the raw HEX strings so we can confirm the parser can see them.

In [ ]:
from pathlib import Path
import re

raw_dir = Path("../data/raw")
if not raw_dir.exists():
    raw_dir = Path("data/raw")  # run from repo root

log_files = [f for f in raw_dir.iterdir() if f.suffix.lower() in (".txt", ".csv") and f.name != "README.md"]
log_files.sort(key=lambda p: p.name)
log_file = log_files[0] if log_files else None
print(f"Using log: {log_file}" if log_file else "No .txt/.csv in data/raw/")
print(f"Files in data/raw: {[f.name for f in raw_dir.iterdir() if f.is_file()]}")

In [ ]:
if log_file:
    lines = log_file.read_text(encoding="utf-8", errors="replace").splitlines()
    first_100 = lines[:100]
    print(f"Total lines in file: {len(lines)}")
    print(f"First 100 lines:\n")
    for i, line in enumerate(first_100, 1):
        print(f"{i:3d} | {line[:120]}{'...' if len(line) > 120 else ''}")
else:
    print("No log file to load.")

## Raw HEX strings the parser would see

Extract and show the HEX payload from lines that look like BLE notifications (3A0FF001 PPG, `value: (0x)`, or `notified: <...>`).

In [ ]:
if not log_file:
    raise SystemExit("No log file.")

lines = log_file.read_text(encoding="utf-8", errors="replace").splitlines()[:100]

def get_hex_snippet(line: str):
    """Return (source, hex_preview) for lines that contain BLE payload HEX."""
    line_lower = line.lower()
    if "3a0ff001" not in line_lower and "3a0ff002" not in line_lower and "3a0ff003" not in line_lower:
        return None
    # nRF Android: value: (0x) A7-1F-00-00-...
    if "value: (0x)" in line:
        start = line.find("value: (0x)") + len("value: (0x)")
        rest = line[start:].strip()
        return ("value: (0x)", rest[:200] + ("..." if len(rest) > 200 else ""))
    # iOS/dash: notified: <ad0c0000 00000000 ...>
    if "notified:" in line and "<" in line and ">" in line:
        start = line.find("<") + 1
        end = line.find(">", start)
        if start > 0 and end > start:
            hex_part = line[start:end].strip()
            return ("notified: <...>", hex_part[:200] + ("..." if len(hex_part) > 200 else ""))
    # CSV/bracket: " to F600 0000 ..."
    if " to " in line and "Updated Value" in line:
        start = line.find(" to ") + 4
        rest = line[start:].strip()
        return (" to ", rest[:200] + ("..." if len(rest) > 200 else ""))
    return None

print("Raw HEX snippets (first 100 lines):\n")
count = 0
for i, line in enumerate(lines, 1):
    out = get_hex_snippet(line)
    if out:
        count += 1
        source, hex_preview = out
        print(f"Line {i} [{source}]:")
        print(f"  {hex_preview}")
        print()
print(f"Total lines with HEX payload in first 100: {count}")